In [85]:
import numpy as np
import pandas as pd
import os

In [86]:
dirty_folder = "dirty"
files = [file for file in os.listdir(dirty_folder) if file.endswith(".parquet")]
files

['users.parquet',
 'abandoned_carts.parquet',
 'inventory.parquet',
 'regulars.parquet',
 'orders.parquet']

In [88]:
#Users CLEAN

df = pd.read_parquet(f"{dirty_folder}/users.parquet")

#Drop information about users with no adults 
drop_indexes = df[((df["count_adults"]==0) & (df["count_people"]>0)) |
   ((df["count_adults"]==0) & (df["count_pets"]>0))].index

df_clean = df.copy()
df_clean.drop(index=drop_indexes, inplace=True)


# 95% of rows with missing info about people
# Comparing df.isna() with df.notna(), similar user_segment and user_nuts1 distribution
# Fill every missing value following column distribution

missing_columns = ["count_adults", "count_children", "count_babies", "count_pets", "user_nuts1"]

for column in missing_columns:
    missing_values = df_clean[column].isna()
    s = df_clean[column].value_counts(normalize=True)
    np.random.seed(99)
    df_clean.loc[missing_values, column] = np.random.choice(s.index, size=missing_values.sum(), p=s.values)

missing_values = df_clean["count_people"].isna()

df_clean.loc[missing_values, "count_people"] = df_clean.loc[missing_values, "count_adults"] + df_clean.loc[missing_values, "count_children"] + df_clean.loc[missing_values, "count_babies"]

df_clean.to_parquet(f"clean/users.parquet")


In [95]:
#Users CLEAN
df = pd.read_parquet(f"{dirty_folder}/orders.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8773 entries, 10 to 64538
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              8773 non-null   int64         
 1   user_id         8773 non-null   object        
 2   created_at      8773 non-null   datetime64[us]
 3   order_date      8773 non-null   datetime64[us]
 4   user_order_seq  8773 non-null   int64         
 5   ordered_items   8773 non-null   object        
dtypes: datetime64[us](2), int64(2), object(2)
memory usage: 479.8+ KB
